In [ ]:
from frgpascal.analysis.processing import load_all
import matplotlib.pyplot as plt
from pathlib import Path
import json # load maestro logs
import pandas as pd
import numpy as np

In [ ]:
# load a csv file

path = 'data/20221011_B9-char_1/'
metrics = pd.read_csv(path + 'fitted_characterization_metrics.csv')
metrics

In [ ]:
plt.title('PL Intensity vs eV')
plt.scatter(metrics['pl_peakev_0'], metrics['pl_intensity_0'])
plt.ylabel('pl_intensity_0')
plt.xlabel('pl_peakev_0')

In [ ]:
# visualize the change in metrics over time
# note a line graph only makes sense if the samples are indeed sequential

columns = ['pl_intensity_0', 'pl_peakev_0', 'pl_fwhm_0', 'df_median_0', 'bf_inhomogeneity_0']
nrows=len(columns)

fig, ax = plt.subplots(nrows=nrows, ncols=1, sharex=True, figsize=(7,nrows*1.5))
ax[0].set_title('Fitted Characterization Metrics')

for i in range(nrows):
    plt.subplot(nrows,1,i+1)
    plt.ylabel(columns[i])
    plt.plot(metrics[columns[i]])
plt.xlabel('Sample Number')

In [ ]:
from natsort import index_natsorted

def load_all_sorted(chardir):
	metricdf, rawdf = load_all(
		chardir,
		t_kwargs=dict(
			wlmin=700,
			wlmax=900
		)
	)
	rawdf= rawdf.sort_values(
   		by='name',
   		key=lambda x: np.argsort(index_natsorted(rawdf['name']))
    )
	rawdf = rawdf.reset_index(drop=True)


	metricdf= metricdf.sort_values(
    	by='name',
    	key=lambda x: np.argsort(index_natsorted(metricdf['name']))
    )
	metricdf = metricdf.reset_index(drop=True)
	return metricdf, rawdf

In [ ]:
chardir_0 = 'data/20221011_B9-char_1/Characterization/'
metricdf, rawdf = load_all_sorted(chardir_0)

In [ ]:
metricdf.columns

In [ ]:
# determine what type of data
for c in rawdf.columns:
    print(c, type(rawdf[c][0]))
print('pl_0 keys: ' + str(rawdf['pl_0'][0].keys()))
print('pl_0["wl"] shape: ' + str(rawdf['pl_0'][0]['wl'].shape))
print('pl_0["cps"] shape: ' + str(rawdf['pl_0'][0]['cps'].shape))

print('t_0 keys: ' + str(rawdf['t_0'][0].keys()))
print('t_0["wl"] shape: ' + str(rawdf['t_0'][0]['wl'].shape))
print('t_0["t"] shape: ' + str(rawdf['t_0'][0]['t'].shape))
print('t_0["a"] shape: ' + str(rawdf['t_0'][0]['a'].shape))

print('df_0 shape: ' + str(rawdf['df_0'][0].shape))
print('bf_0 shape: ' + str(rawdf['bf_0'][0].shape))
print('plimg_0 shape: ' + str(rawdf['plimg_0'][0].shape))

rawdf.head(1)

In [ ]:
# plot images

fig, ax = plt.subplots(1,3, figsize=(16,10))
i=20

ax[0].imshow(rawdf['df_0'][i][:,:,0], cmap='gray')
ax[1].imshow(rawdf['bf_0'][i])
ax[2].imshow(rawdf['plimg_0'][i][:,:,0]) # use red channel for PL measurements, since that's what bandgap energy is

In [ ]:
# plot histograms showing image rgb channel intensities

n_bins = 50
fig, ax = plt.subplots(3,3, figsize=(18,10), sharex='col', sharey='col')

imgs = ['df_0','bf_0','plimg_0']
colors = ['tab:red','tab:green','tab:blue']

for x in range(3):
    for y in range(3):
        ax[y,x].set_title(f'{x},{y}')
        ax[y,x].hist(
            rawdf[imgs[x]][i][:,:,y].flatten(),
            bins=n_bins,
            color=colors[y]
            )

In [ ]:
# try overlaying bf/df/pl images to see how it looks

# try each of the 3 possible two-image combinations
# for now the "filter" is simple multiplication
fig, ax = plt.subplots(1,3, figsize=(16,10))

ax[0].set_title('df * bf')
ax[1].set_title('bf * pl')
ax[2].set_title('pl * df')

ax[0].imshow(rawdf['df_0'][i][:,:,0] * rawdf['bf_0'][i][:,:,0]) # df * bf
ax[1].imshow(rawdf['bf_0'][i][:,:,0] * rawdf['plimg_0'][i][:,:,0]) # bf * pl
ax[2].imshow(rawdf['plimg_0'][i][:,:,0] * rawdf['df_0'][i][:,:,0]) # pl * df

In [ ]:
# plot the pl and transmission spectra

fig, ax_pl = plt.subplots(figsize=(8,6))
ax_pl.plot(rawdf['pl_0'][i]['wl'], rawdf['pl_0'][i]['cps'])
ax_pl.set_xlabel('Wavelength [nm]')
ax_pl.set_ylabel('Intensity [cps]')

ax_t = ax_pl.twinx()
ax_t.set_ylabel('Transmission/Absorption')

ax_t.set_ylim(0, 3)
ax_t.plot(rawdf['t_0'][i]['wl'], rawdf['t_0'][i]['t'], color='tab:orange')
ax_t.plot(rawdf['t_0'][i]['wl'], rawdf['t_0'][i]['a'], color='tab:green')

fig.legend(['PL', 'T', 'A'])
plt.title('PL and Transmission Spectra')